In [1]:
import sys
import os
sys.path.append('C:\\Users\\luoyan011\\Desktop\\PersonalLearning\\GitHub\\python_functions\\jl_nlp_pkg')
sys.path.append('C:\\Users\\luoyan011\\Desktop\\PersonalLearning\\GitHub\\python_functions\\jl_model_explain_pkg')
import nlpbasic.textClean as textClean
import nlpbasic.docVectors as DocVector
import nlpbasic.dataExploration as DataExploration
import nlpbasic.lda as lda
import nlpbasic.tfidf as tfidf

import model_explain.plot as meplot
# import model_explain.shap as meshap
import tensorflow as tf
from numpy import array,asarray,zeros
from scipy.spatial import distance
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np
import re
root_path = 'C:\\Users\\luoyan011\\Desktop\\PersonalLearning\\GitHub\\NLP_data'

In [2]:
def de_emojify(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')
def tweet_proc(df, text_col='text'):
    df['orig_text'] = df[text_col]
    # Remove twitter handles
    df[text_col] = df[text_col].apply(lambda x:re.sub('@[^\s]+','',x))
    # Remove URLs
    df[text_col] = df[text_col].apply(lambda x:re.sub(r"http\S+", "", x))
    # Remove emojis
    df[text_col] = df[text_col].apply(de_emojify)
    # Remove hashtags
    df[text_col] = df[text_col].apply(lambda x:re.sub(r'\B#\S+','',x))
    return df[df[text_col]!='']

In [13]:
covid_tweet = pd.read_csv(os.path.join(root_path, "Covid-19 Twitter Dataset (Aug-Sep 2020).csv"))
covid_tweet = covid_tweet[covid_tweet.original_text.isnull()==False].drop_duplicates().reset_index(drop=True)
covid_tweet = tweet_proc(covid_tweet,'original_text')
covid_tweet['label'] = np.nan
covid_tweet = covid_tweet[covid_tweet.lang=='en']
covid_tweet = covid_tweet[['id', 'original_text', 'sentiment', 'label']].rename(columns={'original_text':'text'})
covid_tweet.head(3)

,id,text,sentiment,label
0,1.300000e+18,RT 91-year-old Ex-Vice President Moody Awori ...,neu,NaN
1,1.300000e+18,RT BREAKING: The Department of Health reports...,neu,NaN
2,1.300000e+18,RT Helps Out Fan Who Requested Him To Help A...,pos,NaN


In [26]:
basic_tweet = pd.read_csv(os.path.join(root_path, "sentiment140_twitter.csv"), names=['target', 'id', 'date', 'flag', 'user', 'text'], header=None)
basic_tweet = basic_tweet[basic_tweet.text.isnull()==False].drop_duplicates().reset_index(drop=True)
basic_tweet = tweet_proc(basic_tweet,'text')
basic_tweet['label'] = np.where(basic_tweet['target']==0, 0, 1)
basic_tweet['sentiment'] = np.where(basic_tweet['target']==0, 'neg', 'pos')
basic_tweet = basic_tweet[['id', 'text', 'sentiment', 'label']]
basic_tweet.head(3)

,id,text,sentiment,label
0,1467810369,"- Awww, that's a bummer. You shoulda got Da...",neg,0
1,1467810672,is upset that he can't update his Facebook by ...,neg,0
2,1467810917,I dived many times for the ball. Managed to s...,neg,0


In [27]:

df_lm = basic_tweet.append(covid_tweet)
df_clas = df_lm.dropna(subset=['label'])
print(len(df_lm), len(df_clas))

1842736 1600000


## DL & Transfer Learning with fastai
some reference:

https://www.kaggle.com/maroberti/fastai-with-transformers-bert-roberta

https://www.kaggle.com/twhelan/covid-19-vaccine-sentiment-analysis-with-fastai
